**TEXT SUMMARIZATION**

Text summarization is the process of condensing a larger piece of text into a shorter version while retaining its key information and main points. It's done using algorithms that analyze and extract important content, providing a concise summary that captures the essence of the original text.


In [ ]:
# Importing the required Libraries
import numpy as np
import pandas as pd
import nltk

import re

import matplotlib.pyplot as plt

from nltk.tokenize import sent_tokenize

from nltk.corpus import stopwords

from sklearn.metrics.pairwise import cosine_similarity

import networkx as nx


In [ ]:
# Extract word vectors
word_embeddings = {}
file = open('glove.6B.100d.txt', encoding='utf-8')
for line in file:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
file.close()
len(word_embeddings)


3672

In [ ]:
# reading the file
df = pd.read_excel('TASK.xlsx')
df


,TEST DATASET,Unnamed: 1
0,NaN,Introduction
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


In [ ]:
df.columns

Index(['TEST DATASET', 'Unnamed: 1'], dtype='object')

In [ ]:
df.rename(columns = {'Unnamed: 1' : 'Introduction' }, inplace=True)


In [ ]:
df.drop(0)

,TEST DATASET,Introduction
1,NaN,Acnesol Gel is an antibiotic that fights bacte...
2,NaN,Ambrodil Syrup is used for treating various re...
3,NaN,Augmentin 625 Duo Tablet is a penicillin-type ...
4,NaN,Azithral 500 Tablet is an antibiotic used to t...
5,NaN,Alkasol Oral Solution is a medicine used in th...
...,...,...
996,NaN,Azapure Tablet belongs to a group of medicines...
997,NaN,Arimidex 1mg Tablet is used alone or with oth...
998,NaN,Arpimune ME 100mg Capsule is used to prevent y...
999,NaN,Amlodac CH Tablet is a combination medicine us...


In [ ]:
# Converting the DataFrame into a dictionary
text_dictionary = {}
for i in range(1,len(df['TEST DATASET'])):
    text_dictionary[i] = df['Introduction'][i]

print(text_dictionary[1])


Acnesol Gel is an antibiotic that fights bacteria. It is used to treat acne, which appears as spots or pimples on your face, chest or back. This medicine works by attacking the bacteria that cause these pimples.Acnesol Gel is only meant for external use and should be used as advised by your doctor. You should normally wash and dry the affected area before applying a thin layer of the medicine. It should not be applied to broken or damaged skin. Avoid any contact with your eyes, nose, or mouth. Rinse it off with water if you accidentally get it in these areas. It may take several weeks for your symptoms to improve, but you should keep using this medicine regularly. Do not stop using it as soon as your acne starts to get better. Ask your doctor when you should stop treatment.Common side effects like minor itching, burning, or redness of the skin and oily skin may be seen in some people. These are usually temporary and resolve on their own. Consult your doctor if they bother you or do not

In [ ]:
#There are 1000 such description of the different medicines. The task is to give summarised form of these description.



In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
    stop_words = stopwords.words('english')

    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new


In [ ]:
# function to make vectors out of the sentences

def sentence_vector_func (sentences_cleaned) :
    sentence_vector = []
    for i in sentences_cleaned:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vector.append(v)

    return (sentence_vector)


In [ ]:
# function to get the summary of the articles

def summary_text (test_text, n = 5):
    sentences = []

    # tokenising the text
    sentences.append(sent_tokenize(test_text))
    # print(sentences)
    sentences = [y for x in sentences for y in x] # flatten list
    # print(sentences)

    # remove punctuations, numbers and special characters
    clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")

    # make alphabets lowercase
    clean_sentences = [s.lower() for s in clean_sentences]
    #print(clean_sentences)


    # remove stopwords from the sentences
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    #print(clean_sentences)

    sentence_vectors = sentence_vector_func(clean_sentences)

    # similarity matrix
    sim_mat = np.zeros([len(sentences), len(sentences)])
    #print(sim_mat)

    # Finding the similarities between the sentences
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]


    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph)
    #print(scores)

    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)))
    # Extract sentences as the summary
    summarised_string = ''
    for i in range(n):

        try:
            summarised_string = summarised_string + str(ranked_sentences[i][1])
        except IndexError:
            print ("Summary Not Available")

    return (summarised_string)


In [ ]:
pip install nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print("Kindly let me know in how many sentences you want the summary - ")
x = int(input())

summary_dictionary = {}

for key in text_dictionary:

    para = text_dictionary[key]
    print("Summary of the article - ",key)
    summary = summary_text(para,x)
    summary_dictionary[key] = summary

    print(summary)
    print('='*120)

print ("*"*40,"The process has been completed successfully","*"*40)


Kindly let me know in how many sentences you want the summary - 
3
Summary of the article -  1
Acnesol Gel is an antibiotic that fights bacteria.It should not be applied to broken or damaged skin.Consult your doctor about using this medicine if you are pregnant or breastfeeding.
Summary of the article -  2
Ambrodil Syrup is used for treating various respiratory tract disorders associated with excessive mucus.For better results, it is suggested to take it at the same time every day.Your doctor will decide how much you need to improve your symptoms.
Summary of the article -  3
It is used to treat infections of the lungs (e.g., pneumonia), ear, nasal sinus, urinary tract, skin and soft tissue.Augmentin 625 Duo Tablet is a penicillin-type of antibiotic that helps your body fight infections caused by bacteria.This medicine is generally regarded as safe to use during pregnancy and breastfeeding if prescribed by a doctor.
Summary of the article -  4
These are usually temporary and subside wit

<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.r

It thins mucus in the nose, windpipe and lungs, making it easier to cough out.It is beneficial to have plenty of fluids while taking this medication.You should also tell your doctor if you have any kidney or liver diseases so that your doctor can prescribe a suitable dose for you.
Summary of the article -  8
This medicine should relieve indigestion and heartburn within a few hours.Follow the advice of your doctor while taking this medicine.Aciloc 150 Tablet is a medicine that reduces the amount of excess acid make by your stomach.
Summary of the article -  9
Inform your doctor if you are pregnant, or breastfeeding.Avoid driving or attention-seeking activity.Generally, it is advised to avoid alcohol while on treatment.
Summary of the article -  10
These are usually temporary and subside with the completion of treatment.Pregnant or breastfeeding women should consult their doctor before using this medicine.Do not skip any doses and finish the full course of treatment even if you feel bett

<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.r

Occasionally it can cause thinning of your hair.It is used to treat fungal infections of the skin.Arcolane 2% Scalp Solution belongs to a group of medicines called antifungals.
Summary of the article -  13
Inform your doctor if you are pregnant, or breastfeeding.Atarax 25mg Tablet is an antihistamine used to treat symptoms of skin allergy like itching, swelling, and rashes.Generally, it is advised to avoid alcohol while on treatment.
Summary of the article -  14
Altraday Capsule SR is a combination of two medicines.You should take it as advised by your doctor.The dose will depend on what you are taking it for and how well it helps your symptoms.
Summary of the article -  15
You should always wash your hands after applying it.Avoid getting it into your eyes or mouth.It is for external use only.
Summary of the article -  16
It also relaxes the muscles in your airway.It thins mucus in the nose, windpipe and lungs making it easier to cough out.Ambrodil-S Syrup is a combination medicine use

...

<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.r

It is prescribed to treat various types of bacterial infections.Ampitrust 1.5gm Injection is a combination of two antibiotics.It fights against the infection by killing the microorganisms.Ampitrust 1.5gm Injection is generally administered by a healthcare professional and should not be self-administered.
Summary of the article -  996
Azapure Tablet belongs to a group of medicines called immunosuppressants.Follow your doctor’s instructions on this.Swallow it as a whole,  do not crush, chew, break, or open them.
Summary of the article -  997
Swallow the tablets whole with a drink of water.Tell your doctor if you are pregnant or breastfeeding.It should be taken as your doctor's advice.
Summary of the article -  998
Your doctor will plan the correct dose for you.However, it is not recommended while breastfeeding.The results of tests may lead to changes in your dose.
Summary of the article -  999


<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")


Drink plenty of fluids while taking medicine to overcome muscle weakness, dry mouth, and extreme thirst.It should be taken in the morning with food to avoid frequent urination at night.Amlodac CH Tablet should be taken with food preferably in the morning to avoid frequent urination at night.Consult your doctor If any of these bother you, or get worse, or do not go away.
Summary of the article -  1000
It is known as a calcium channel blocker.Pregnant or breastfeeding women should consult their doctor before taking it.It works by relaxing blood vessels to lower blood pressure.
**************************************** The process has been completed successfully ****************************************


<ipython-input-12-0a08ace92b9c>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-z A-Z 0-9]", " ")
